# APRIORI

## ALGORITMO DE ASSOCIAÇÃO

# LIB's

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

import datetime

from apyori import apriori

# DATASET

In [2]:
df = pd.read_csv('data.csv', sep = ';', decimal = ',')
df.head()

,data_pedido,hora_pedido,tipo_entrega,tempo_minutos,borda,refrigerante,Sobremesa,valor_pizza,valor_total
0,29/10/2011,21:00:00,Entrega,55,N,N,S,38.5,38.5
1,29/10/2011,21:10:00,Entrega,45,N,N,S,38.5,38.5
2,30/10/2011,20:17:00,Entrega,51,N,N,S,38.5,38.5
3,01/11/2011,21:10:00,Entrega,50,N,S,S,34.0,38.5
4,04/11/2011,20:37:00,Entrega,33,N,S,S,34.0,38.5


In [3]:
df.shape

(1000, 9)

In [4]:
df.dtypes

data_pedido       object
hora_pedido       object
tipo_entrega      object
tempo_minutos      int64
borda             object
refrigerante      object
Sobremesa         object
valor_pizza      float64
valor_total      float64
dtype: object

## VARIÁVEIS DE TEMPO

In [5]:
# alterando o tipo de dado da variavel data_pedido
df['data_pedido'] = pd.to_datetime(df['data_pedido'], format = ('%d/%m/%Y'))

# criando coluna dia da semana
df['dia_semana'] = df['data_pedido'].dt.day_name()

In [6]:
# horário dos pedidos
print(df['hora_pedido'].min(), df['hora_pedido'].max())

18:00:00 23:42:00


In [7]:
# 'extraindo' somente a hora
df['horario'] = df['hora_pedido'].str.slice(start=0, stop = 2)

#contando a qt de ocorrencias
df['horario'].value_counts()

20    315
21    241
19    166
22    130
18    101
23     47
Name: horario, dtype: int64

In [8]:
# notamos que a pizzaria funciona por 6 horas. Dividindo por cada duas horas,
# podemos categorizar os horários como horário de pico alto, medio e baixo

pico_dict = ({ '20' : 'pico - alto',
             '21' : 'pico - alto',
             '19' : 'pico - medio',
             '22' : 'pico - medio',
             '18' : 'pico - baixo',
             '23' : 'pico - baixo'})

df['horario'] = df['horario'].map(pico_dict)

df['horario'].value_counts()

pico - alto     556
pico - medio    296
pico - baixo    148
Name: horario, dtype: int64

## VARIÁVEL 'BORDA'

In [9]:
df['borda'].value_counts()

N          849
S          126
Borda S     18
Borda N      7
Name: borda, dtype: int64

In [10]:
# tratando a variável borda
df['borda'] = df['borda'].apply(lambda x: 'N' if x == 'Borda N'
                 else x)

df['borda'] = df['borda'].apply(lambda x: 'S' if x == 'Borda S'
                 else x)

# REFRIGERANTE, BORDA E SOBREMESA

In [11]:
df['borda'] = df['borda'].apply(lambda x: 'borda N' if x == 'N'
                 else x)

df['borda'] = df['borda'].apply(lambda x: 'borda S' if x == 'S'
                 else x)

In [12]:
df['refrigerante'] = df['refrigerante'].apply(lambda x: 'refrigerante N' if x == 'N'
                 else x)

df['refrigerante'] = df['refrigerante'].apply(lambda x: 'refrigerante S' if x == 'S'
                 else x)

In [13]:
df['Sobremesa'] = df['Sobremesa'].apply(lambda x: 'sobremesa N' if x == 'N'
                 else x)

df['Sobremesa'] = df['Sobremesa'].apply(lambda x: 'sobremesa S' if x == 'S'
                 else x)

## VALOR DA PIZZA

In [14]:
print(df['valor_pizza'].min(), df['valor_pizza'].max() )

10.0 41.0


In [15]:
df['range_valor_pizza'] = pd.cut( x = df['valor_pizza'], bins = 5, include_lowest = True)

In [16]:
for f in df['range_valor_pizza'].unique():
    print(f)

(34.8, 41.0]
(28.6, 34.8]
(22.4, 28.6]
(16.2, 22.4]
(9.968, 16.2]


In [17]:
df.head()

,data_pedido,hora_pedido,tipo_entrega,tempo_minutos,borda,refrigerante,Sobremesa,valor_pizza,valor_total,dia_semana,horario,range_valor_pizza
0,2011-10-29,21:00:00,Entrega,55,borda N,refrigerante N,sobremesa S,38.5,38.5,Saturday,pico - alto,"(34.8, 41.0]"
1,2011-10-29,21:10:00,Entrega,45,borda N,refrigerante N,sobremesa S,38.5,38.5,Saturday,pico - alto,"(34.8, 41.0]"
2,2011-10-30,20:17:00,Entrega,51,borda N,refrigerante N,sobremesa S,38.5,38.5,Sunday,pico - alto,"(34.8, 41.0]"
3,2011-11-01,21:10:00,Entrega,50,borda N,refrigerante S,sobremesa S,34.0,38.5,Tuesday,pico - alto,"(28.6, 34.8]"
4,2011-11-04,20:37:00,Entrega,33,borda N,refrigerante S,sobremesa S,34.0,38.5,Friday,pico - alto,"(28.6, 34.8]"


# APRIORI I

In [18]:
# Selecionando as colunas
df1 = df[['tipo_entrega', 'borda', 'refrigerante', 'Sobremesa', 'dia_semana', 'horario', 'range_valor_pizza']]

df1.head()

,tipo_entrega,borda,refrigerante,Sobremesa,dia_semana,horario,range_valor_pizza
0,Entrega,borda N,refrigerante N,sobremesa S,Saturday,pico - alto,"(34.8, 41.0]"
1,Entrega,borda N,refrigerante N,sobremesa S,Saturday,pico - alto,"(34.8, 41.0]"
2,Entrega,borda N,refrigerante N,sobremesa S,Sunday,pico - alto,"(34.8, 41.0]"
3,Entrega,borda N,refrigerante S,sobremesa S,Tuesday,pico - alto,"(28.6, 34.8]"
4,Entrega,borda N,refrigerante S,sobremesa S,Friday,pico - alto,"(28.6, 34.8]"


In [19]:
## Tranformando o DataFrame em uma lista

dados = []

## range "ap" == tamanho do dataframe
## 0, 6 == colunas
for i in range(len(df1)):
    dados.append([str(df1.values[i, j]) for j in range(0, 7)])


## Configurando os parametros
## Suporte == 20%
    regras = apriori(dados, min_support = 0.02, min_confidence = 0.2, min_lift = 2, min_lenght = 2)

result = list(regras)
result2 = [list(x) for x in result]


In [20]:
df2 = []

for j in range(0,7): ## quantidade de regras
    df2.append([list(x) for x in result2[j][2]])
    
df2

[[[frozenset({'(34.8, 41.0]'}),
   frozenset({'sobremesa S'}),
   0.48936170212765956,
   7.303906001905366],
  [frozenset({'sobremesa S'}),
   frozenset({'(34.8, 41.0]'}),
   0.34328358208955223,
   7.303906001905367]],
 [[frozenset({'borda S'}),
   frozenset({'sobremesa S'}),
   0.2430555555555556,
   3.6276948590381433],
  [frozenset({'sobremesa S'}),
   frozenset({'borda S'}),
   0.5223880597014926,
   3.6276948590381433]],
 [[frozenset({'refrigerante S'}),
   frozenset({'sobremesa S'}),
   0.2803030303030303,
   4.1836273179556756],
  [frozenset({'sobremesa S'}),
   frozenset({'refrigerante S'}),
   0.5522388059701492,
   4.1836273179556756]],
 [[frozenset({'(16.2, 22.4]'}),
   frozenset({'Buscar', 'pico - medio'}),
   0.2596685082872928,
   2.0608611768832765],
  [frozenset({'Buscar', 'pico - medio'}),
   frozenset({'(16.2, 22.4]'}),
   0.373015873015873,
   2.0608611768832765]],
 [[frozenset({'(16.2, 22.4]', 'refrigerante S'}),
   frozenset({'Buscar'}),
   0.9523809523809523,
  

# APRIORI II

In [21]:
# treinando sem a variável sobremesa
# Selecionando as colunas
df1 = df[['tipo_entrega', 'borda', 'refrigerante', 'dia_semana', 'horario', 'range_valor_pizza']]

In [22]:
dados = []

for i in range(len(df1)):
    dados.append([str(df1.values[i, j]) for j in range(0, 6)])
    regras = apriori(dados, min_support = 0.02, min_confidence = 0.2, min_lift = 2, min_lenght = 2)

result = list(regras)
result2 = [list(x) for x in result]


In [23]:
df2 = []
for j in range(0,4): ## quantidade de regras
    df2.append([list(x) for x in result2[j][2]])
    
df2

[[[frozenset({'(16.2, 22.4]'}),
   frozenset({'Buscar', 'pico - medio'}),
   0.2596685082872928,
   2.0608611768832765],
  [frozenset({'Buscar', 'pico - medio'}),
   frozenset({'(16.2, 22.4]'}),
   0.373015873015873,
   2.0608611768832765]],
 [[frozenset({'(16.2, 22.4]', 'refrigerante S'}),
   frozenset({'Buscar'}),
   0.9523809523809523,
   2.2461814914645104],
  [frozenset({'Buscar', 'refrigerante S'}),
   frozenset({'(16.2, 22.4]'}),
   0.5128205128205129,
   2.8332625017707893]],
 [[frozenset({'(16.2, 22.4]'}),
   frozenset({'Buscar', 'borda N', 'pico - medio'}),
   0.2430939226519337,
   2.2933388929427707],
  [frozenset({'(16.2, 22.4]', 'borda N'}),
   frozenset({'Buscar', 'pico - medio'}),
   0.26347305389221554,
   2.091055983271552],
  [frozenset({'Buscar', 'pico - medio'}),
   frozenset({'(16.2, 22.4]', 'borda N'}),
   0.3492063492063492,
   2.091055983271552],
  [frozenset({'Buscar', 'borda N', 'pico - medio'}),
   frozenset({'(16.2, 22.4]'}),
   0.4150943396226415,
   2.293

# APRIORI III

In [27]:
# treinando sem a variável borda
# Selecionando as colunas
df1 = df[['tipo_entrega', 'refrigerante', 'dia_semana', 'horario', 'range_valor_pizza']]

In [28]:
dados = []

for i in range(len(df1)):
    dados.append([str(df1.values[i, j]) for j in range(0, 5)])
    regras = apriori(dados, min_support = 0.02, min_confidence = 0.2, min_lift = 2, min_lenght = 2)

result = list(regras)
result2 = [list(x) for x in result]


In [31]:
df2 = []
for j in range(0,2): ## quantidade de regras
    df2.append([list(x) for x in result2[j][2]])
    
df2

[[[frozenset({'(16.2, 22.4]'}),
   frozenset({'Buscar', 'pico - medio'}),
   0.2596685082872928,
   2.0608611768832765],
  [frozenset({'Buscar', 'pico - medio'}),
   frozenset({'(16.2, 22.4]'}),
   0.373015873015873,
   2.0608611768832765]],
 [[frozenset({'(16.2, 22.4]', 'refrigerante S'}),
   frozenset({'Buscar'}),
   0.9523809523809523,
   2.2461814914645104],
  [frozenset({'Buscar', 'refrigerante S'}),
   frozenset({'(16.2, 22.4]'}),
   0.5128205128205129,
   2.8332625017707893]]]

## RESULTADO

Principais resultados:

***

1. Pizzas a partir de 34.8 --- Sobremesa

2. Borda --- Sobremesa

3. Refrigerante --- Sobremesa

4. Buscar --- Pizza 16.2, 22.4 --- Refrigerante

***

